In [9]:
%%html
<h1 style="color:orange;">INTRO TO BLOCK CHAIN ENGINEERING - PROJECT-9 Flask Ngrok Web app - 12613141</h1>

Installing Libraries required pyngrok and flask

---



In [10]:
!pip install pyngrok

In [11]:
from flask import Flask
from pyngrok import ngrok

Creating project folder structure for flask

---



In [12]:
!mkdir flask_blog
!mkdir flask_blog/static
!mkdir flask_blog/templates
!mkdir flask_blog/static/css

mkdir: cannot create directory ‘flask_blog’: File exists
mkdir: cannot create directory ‘flask_blog/static’: File exists
mkdir: cannot create directory ‘flask_blog/templates’: File exists
mkdir: cannot create directory ‘flask_blog/static/css’: File exists





---


1.  Creating styles of css extension and storing in static folder
2.  Creating html pages of html extension and storing in templates folder
3.  Creating schema file sql file
4.  Creating database using sqlite with schema.sql file and storing posts


---








In [13]:
%%writefile  flask_blog/static/style.css
h1 {
    border: 2px #eee solid;
    color: brown;
    text-align: center;
    padding: 10px;
}

Writing flask_blog/static/style.css


In [14]:
%%writefile flask_blog/templates/base.html
<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">

    <title>{% block title %} {% endblock %}</title>
  </head>
  <body>
      <nav class="navbar navbar-expand-md navbar-light bg-light">
        <a class="navbar-brand" href="{{ url_for('index')}}">FlaskBlog</a>
        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNav" aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
            <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse" id="navbarNav">
            <ul class="navbar-nav">
            <li class="nav-item">
                <a class="nav-link" href="#">About</a>
            </li>
            <li class="nav-item">
                <a class="nav-link" href="{{url_for('create')}}">New Post</a>
            </li>
            </ul>
        </div>
      </nav>
    <div class="container">
        {% for message in get_flashed_messages() %}
            <div class="alert alert-danger">{{ message }}</div>
        {% endfor %}
        {% block content %} {% endblock %}
    </div>

    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js" integrity="sha384-UO2eT0CpHqdSJQ6hJty5KVphtPhzWj9WO1clHTMGa3JDZwrnQq4sF86dIHNDz0W1" crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js" integrity="sha384-JjSmVgyd0p3pXB1rRibZUAYoIIy6OrQ6VrjIEaFf/nJGzIxFDsf4x0xIM+B07jRM" crossorigin="anonymous"></script>
  </body>
</html>


Writing flask_blog/templates/base.html


In [15]:
%%writefile flask_blog/templates/create.html
{% extends 'base.html' %}

{% block content %}
<h1>{% block title %} Create a New Post {% endblock %}</h1>

<form method="post">
    <div class="form-group">
        <label for="title">Title</label>
        <input type="text" name="title"
               placeholder="Post title" class="form-control"
               value="{{ request.form['title'] }}"></input>
    </div>

    <div class="form-group">
        <label for="content">Content</label>
        <textarea name="content" placeholder="Post content"
                  class="form-control">{{ request.form['content'] }}</textarea>
    </div>
    <div class="form-group">
        <button type="submit" class="btn btn-primary">Submit</button>
    </div>
</form>
{% endblock %}

Writing flask_blog/templates/create.html


In [16]:
%%writefile flask_blog/templates/edit.html
{% extends 'base.html' %}

{% block content %}
<h1>{% block title %} Edit "{{ post['title'] }}" {% endblock %}</h1>

<form method="post">
    <div class="form-group">
        <label for="title">Title</label>
        <input type="text" name="title" placeholder="Post title"
               class="form-control"
               value="{{ request.form['title'] or post['title'] }}">
        </input>
    </div>

    <div class="form-group">
        <label for="content">Content</label>
        <textarea name="content" placeholder="Post content"
                  class="form-control">{{ request.form['content'] or post['content'] }}</textarea>
    </div>
    <div class="form-group">
        <button type="submit" class="btn btn-primary">Submit</button>
    </div>
</form>
<hr>
<form action="{{ url_for('delete', id=post['id']) }}" method="POST">
    <input type="submit" value="Delete Post"
            class="btn btn-danger btn-sm"
            onclick="return confirm('Are you sure you want to delete this post?')">
</form>
{% endblock %}



Writing flask_blog/templates/edit.html


In [17]:
%%writefile flask_blog/templates/index.html
{% extends 'base.html' %}

{% block content %}
    <h1>{% block title %} Welcome to FlaskBlog {% endblock %}</h1>
    {% for post in posts %}
        <a href="{{ url_for('post', post_id=post['id']) }}">
            <h2>{{ post['title'] }}</h2>
        </a>
        <span class="badge badge-primary">{{ post['created'] }}</span>
        <a href="{{ url_for('edit', id=post['id']) }}">
            <span class="badge badge-warning">Edit</span>
        </a>
        <hr>
    {% endfor %}
{% endblock %}

Writing flask_blog/templates/index.html


In [18]:
%%writefile flask_blog/templates/post.html
{% extends 'base.html' %}

{% block content %}
    <h2>{% block title %} {{ post['title'] }} {% endblock %}</h2>
    <span class="badge badge-primary">{{ post['created'] }}</span>
    <p>{{ post['content'] }}</p>
{% endblock %}

Writing flask_blog/templates/post.html


In [19]:
%%writefile flask_blog/schema.sql
DROP TABLE IF EXISTS posts;

CREATE TABLE posts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    title TEXT NOT NULL,
    content TEXT NOT NULL
);

Writing flask_blog/schema.sql


Connecting to database.db and creating posts table using schema.sql scripts
Inserting dummy posts by making and connection and closing it



---



In [20]:
import sqlite3

connection = sqlite3.connect('flask_blog/database.db')

with open('flask_blog/schema.sql') as f:
    connection.executescript(f.read())

cur = connection.cursor()

cur.execute("INSERT INTO posts (title, content) VALUES (?, ?)",
            ('First Post', 'Content for the first post')
            )

cur.execute("INSERT INTO posts (title, content) VALUES (?, ?)",
            ('Second Post', 'Content for the second post')
            )

connection.commit()
connection.close()

Defining a port number on which our web server blog runs


---



In [21]:
port_no = 5000

Instantiating an app using Flask and ngrok with auth_token, and app is configured with basic url and secret key
Generating a public URL using ngrok
Creating routes for html templates to perform crud operations for the blog
Create, read, update, delete operations on posts.

---



In [25]:
import sqlite3
from flask import Flask, render_template, request, url_for, flash, redirect
from werkzeug.exceptions import abort
from pyngrok import ngrok

def get_db_connection():
    conn = sqlite3.connect('flask_blog/database.db')
    conn.row_factory = sqlite3.Row
    return conn


def get_post(post_id):
    conn = get_db_connection()
    post = conn.execute('SELECT * FROM posts WHERE id = ?',
                        (post_id,)).fetchone()
    conn.close()
    if post is None:
        abort(404)
    return post

#Defining a flask a
app = Flask(__name__, static_folder="flask_blog/static", template_folder="flask_blog/templates")
ngrok.set_auth_token("2ebbESDbvFrGryxHs1U7tpDZ2SY_3mxQiSvWov2wQvovyDHeH")
public_url =  ngrok.connect(port_no).public_url
app.config["SECRET_KEY"] = "2ebbESDbvFrGryxHs1U7tpDZ2SY_3mxQiSvWov2wQvovyDHeH"

@app.route('/')
def index():
    conn = get_db_connection()
    posts = conn.execute('SELECT * FROM posts').fetchall()
    conn.close()
    return render_template('index.html', posts=posts)


@app.route('/<int:post_id>')
def post(post_id):
    post = get_post(post_id)
    return render_template('post.html', post=post)


@app.route('/create', methods=('GET', 'POST'))
def create():
    if request.method == 'POST':
        title = request.form['title']
        content = request.form['content']

        if not title:
            flash('Title is required!')
        else:
            conn = get_db_connection()
            conn.execute('INSERT INTO posts (title, content) VALUES (?, ?)',
                         (title, content))
            conn.commit()
            conn.close()
            return redirect(url_for('index'))

    return render_template('create.html')


@app.route('/<int:id>/edit', methods=('GET', 'POST'))
def edit(id):
    post = get_post(id)

    if request.method == 'POST':
        title = request.form['title']
        content = request.form['content']

        if not title:
            flash('Title is required!')
        else:
            conn = get_db_connection()
            conn.execute('UPDATE posts SET title = ?, content = ?'
                         ' WHERE id = ?',
                         (title, content, id))
            conn.commit()
            conn.close()
            return redirect(url_for('index'))

    return render_template('edit.html', post=post)


@app.route('/<int:id>/delete', methods=('POST',))
def delete(id):
    post = get_post(id)
    conn = get_db_connection()
    conn.execute('DELETE FROM posts WHERE id = ?', (id,))
    conn.commit()
    conn.close()
    flash('"{}" was successfully deleted!'.format(post['title']))
    return redirect(url_for('index'))

print(f"To acces the Gloable link please click {public_url}")
app.run(port=port_no)

To acces the Gloable link please click https://8618-34-73-147-117.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:28] "GET /1/edit HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:37] "POST /1/edit HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:46] "GET /2/edit HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:57] "POST /2/edit HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:29:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:30:00] "GET /3/edit HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 22:30:05] "POST /3/delete HTTP/1.1" 302 -
INFO:werkzeug:1